In [9]:
import sys
!{sys.executable} -m pip install pysimplegui
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import calendar
import PySimpleGUI as sg

# Function to handle the setup process
def setup_budget():
    df = pd.read_csv('Financial-Tracker.csv')
    global needs_list, wants_list, savings_list, setupcounter
    ready = sg.popup_yes_no("Are you ready to set up?")

    if ready == "Yes":
        initial_date = datetime.now()
        initial_budget = int(sg.popup_get_text("What was your budget at the beginning of the day?"))
        initial_income = int(sg.popup_get_text("How much did you earn today?"))

        sg.popup("\nPlease let us know more about your spending habits:")
        sum_of_needs = 0
        for i in range(len(needs_list)):
            needs_value = int(sg.popup_get_text("How much did you spend on " + needs_list[i] + " today?"))
            sum_of_needs += needs_value
            monthlyneeds[i] += needs_value
        initial_needs = sum_of_needs

        sum_of_wants = 0
        for i in range(len(wants_list)):
            wants_value = int(sg.popup_get_text("How much did you spend on " + wants_list[i] + " today?"))
            sum_of_wants += wants_value
            monthlywants[i] += wants_value
        initial_wants = sum_of_wants

        sg.popup("\nPlease let us know more about your saving habits:")
        sum_of_savings = 0
        for i in range(len(savings_list)):
            savings_value = int(sg.popup_get_text("How much did you " + savings_list[i] + " today?"))
            sum_of_savings += savings_value
            monthlysavings[i] += savings_value
        initial_savings = sum_of_savings

        initial_expense = initial_needs + initial_wants + initial_savings
        initial_budget = initial_budget + initial_income - initial_expense
        monthsshown.append(nowmonth)
        dailyupdate()

        setupcounter = True
        setup_row = {
            'Date': initial_date,
            'Current_Budget': initial_budget,
            'Income': initial_income,
            'Expense': initial_expense,
            'Needs': initial_needs,
            'Wants': initial_wants,
            'Savings': initial_savings
        }

        df = pd.concat([df, pd.DataFrame(setup_row, index=[0])], ignore_index=True)

    elif ready == "No":
        sg.popup("Sure! Let me know when you're ready to set up!")
        return
    else:
        sg.popup("Error")
        return

    return df

# Function to handle updating data
def update_data(df):
    global needs_list, wants_list, savings_list
    set_date()
    df = pd.read_csv('Financial-Tracker.csv')
    today_date = datetime.now()
    today_income = int(sg.popup_get_text("How much did you earn today?"))

    sg.popup("\nPlease let us know more about how you spent today:")
    sum_of_needs = 0
    for i in range(len(needs_list)):
        needs_value = int(sg.popup_get_text("How much did you spend on " + needs_list[i] + " today?"))
        sum_of_needs += needs_value
        monthlyneeds[i] += needs_value
    today_needs = sum_of_needs

    sum_of_wants = 0
    for i in range(len(wants_list)):
        wants_value = int(sg.popup_get_text("How much did you spend on " + wants_list[i] + " today?"))
        sum_of_wants += wants_value
        monthlywants[i] += wants_value
    today_wants = sum_of_wants

    sg.popup("\nPlease let us know more about your saving habits:")
    sum_of_savings = 0
    for i in range(len(savings_list)):
        savings_value = int(sg.popup_get_text("How much did you " + savings_list[i] + " today?"))
        sum_of_savings += savings_value
        monthlysavings[i] += savings_value
    today_savings = sum_of_savings

    today_expense = today_needs + today_wants + today_savings
    today_budget = df['Current_Budget'].iloc[-1] + today_income - today_expense
    dailyupdate()

    today_row = {
        'Date': today_date,
        'Current_Budget': today_budget,
        'Income': today_income,
        'Expense': today_expense,
        'Needs': today_needs,
        'Wants': today_wants,
        'Savings': today_savings
    }

    df = pd.concat([df, pd.DataFrame(today_row, index=[0])], ignore_index=True)
    return df

# Function to generate insights and show visualizations
def generate_insights(df):
    total_expense = df['Expense'].sum()
    category_expenses = df[['Needs', 'Wants', 'Savings']].sum()

    # Create and display the pie chart
    fig1 = go.Figure(data=[go.Pie(labels=category_expenses.index, values=category_expenses.values)])
    fig1.update_layout(title="Expense Distribution")
    fig1.show()

    # Create and display the bar chart
    fig2 = go.Figure(data=[go.Bar(x=category_expenses.index, y=category_expenses.values)])
    fig2.update_layout(title="Spending Patterns", xaxis_title="Expense Category", yaxis_title="Total Expense")
    fig2.show()

# Function to handle saving goals
def saving_goals(goals):
    name = sg.popup_get_text('Please enter the name of this new goal:')
    datecheck = False
    while not datecheck:
        goaldate = sg.popup_get_text('Please enter the date by which this goal should be reached in the format YYYY-MM-DD:')
        begindate = datetime.now()
        try:
            convertdate = datetime.strptime(goaldate, '%Y-%m-%d')
        except:
            sg.popup("The date inputted has an invalid format.")
            continue
        if convertdate < datetime.now():
            sg.popup("The date inputted is before the current date. Please try again.")
            continue
        else:
            datecheck = True
    money = int(sg.popup_get_text('Please enter the amount of pesos required for this goal:'))
    sg.popup(f"A new goal, {name}, due on {convertdate}, requiring {money} pesos has been set.")
    goals[len(goals)] = {'name': name, 'date': convertdate, 'money': money, 'start': begindate}

# Function to handle notifications
def notifications(df):
    notif = []
    timebenchmarks = [1,2,3,5,10,30]
    spendingbenchmarks = [90,80,70,50,30]
    spendingcheck = False
    monthlyexpense = 0
    for i in range(len(monthlyneeds)):
        monthlyexpense += monthlyneeds[i]
    for i in range(len(monthlywants)):
        monthlyexpense += monthlywants[i]
    for i in range(len(monthlysavings)):
        monthlyexpense += monthlysavings[i]
    for i in range(len(goals)):
        daydiff = goals[i]['date'] - datetime.now()
        notifcheck = False
        for j in range(len(timebenchmarks)):
            if notifcheck == True:
                continue
            if int(daydiff.total_seconds()/86400) < timebenchmarks[j]:
                notif.append(f'You have less than {timebenchmarks[j]} day/s to reach your goal, "{goals[i]["name"]}"')
                notifcheck = True
    for i in range(len(spendingbenchmarks)):
        if spendingcheck == True:
            continue
        if spendinglimit == 0:
            continue
        truespendingmark = spendingbenchmarks[i]/100 * spendinglimit
        if monthlyexpense >= truespendingmark:
            spendingcheck = True
            notif.append(f'You have gone above {spendingbenchmarks[i]}% of your spending limit. There are {monthend - nowday} days left in the month.')
    for i in range(len(notif)):
        sg.popup(notif[i])

# Function to set the date
def set_date():
    global nowday, nowyear, nowmonth, monthinfo, monthend
    nowday = datetime.now().day
    nowyear = datetime.now().year
    nowmonth = datetime.now().month
    monthinfo = calendar.monthrange(nowyear, nowmonth)
    monthend = monthinfo[1]

# Function to set initial variables
def set_vars():
    global needs_list, wants_list, savings_list, goals, spendinglimit, expenses
    needs_list = ["food", "water", "rent/utilities", "clothing", "healthcare"]
    wants_list = ["hobbies", "subscriptions", "shopping for things you don't need"]
    savings_list = ["save", "invest"]
    goals = {}
    spendinglimit = 0
    expenses = 0
    global monthlyneeds, monthlywants, monthlysavings, yearlyvalues, monthsshown
    monthlyneeds = [0] * len(needs_list)
    monthlywants = [0] * len(wants_list)
    monthlysavings = [0] * len(savings_list)
    yearlyvalues = {}
    monthsshown = []

# Function to remove a goal
def choice_goal_remove(df):
    removecheck = False
    while not removecheck:
        removeinput = sg.popup_get_text('Which goal do you want to remove? Otherwise, enter C to cancel.')
        if removeinput == 'C':
            return
        for i in range(len(goals)):
            goalname = goals[i]['name']
            if removeinput == goalname:
                deductinput = sg.popup_get_text('Would you like to automatically make the deduction from your savings? Please input as Y or N.')
                if deductinput == 'Y':
                    newval = df.loc[len(df['Date'])-1,'Savings'] - goals[i]['money']
                    df.loc[len(df['Date'])-1,'Savings'] = newval
                    monthlysavings[0] -= newval
                    sg.popup(f'{goals[i]["money"]} pesos has been deducted.')
                del goals[i]
                removecheck = True
                sg.popup(f'Goal {goalname} has been removed.')
        if not removecheck:
            sg.popup('An error has occurred. Are you sure you put the right name?')

# Function to reset the month's data
def monthreset():
    global yearlyvalues
    monthsshown.append(nowmonth)
    yearlyvalues[nowmonth] = {'needs': monthlyneeds.copy(), 'wants': monthlywants.copy(), 'savings': monthlysavings.copy()}
    reset_lists()

# Function to reset monthly expense lists
def reset_lists():
    global monthlyneeds, monthlywants, monthlysavings
    monthlyneeds = [0] * len(needs_list)
    monthlywants = [0] * len(wants_list)
    monthlysavings = [0] * len(savings_list)

# Function to update daily data
def dailyupdate():
    global yearlyvalues
    yearlyvalues[nowmonth] = {'needs': monthlyneeds.copy(), 'wants': monthlywants.copy(), 'savings': monthlysavings.copy()}

# Function to reset yearly data
def year_reset():
    global yearlyvalues
    yearlyvalues = {}
    reset_lists()
    
# Function to handle goal visualization
def goal_visual(df):
    timespan = int(sg.popup_get_text("How many entries would you like to visualize?"))
    for i in range(len(goals)):
        elapselist = []
        savingslist = []
        goalline = []
        for j in range(timespan):
            try:
                goalline.append(goals[i]['money'])
                elapselist.append(df.loc[len(df['Date']) - 1 - (timespan - j)]['Date'])
                savingslist.append(df.iloc[0:(len(df['Date']) - (timespan - j))]['Savings'].sum())
            except:
                print("An error has occurred.")
                return
        plt.plot(elapselist, savingslist, label='Savings')
        plt.title(f'{goals[i]["name"]} Progress Chart')
        plt.ylabel('Pesos')
        plt.plot(elapselist, goalline, label='Required Amount')
        plt.ylim(0, df['Savings'].sum())
        plt.xlabel(f'Entries (Beginning {timespan} instances ago)')
        plt.legend()
        plt.show()

# Function to handle expense visualization
def expense_visual():
    visinput = sg.popup_get_text("Would you like to analyze expenses on a yearly (Y) or monthly (M) basis?")
    typeinput = sg.popup_get_text("Which category would you like to inspect? Input 'needs', 'wants', or 'savings':")
    if visinput.lower() == 'y':
        valuelist = []
        for j in range(len(yearlyvalues[monthsshown[-1]][typeinput])):
            typesum = 0
            for i in monthsshown:
                typesum += yearlyvalues[i][typeinput][j]
            valuelist.append(typesum)
    elif visinput.lower() == 'm':
        monthinput = int(sg.popup_get_text("Place the number of the month you wish to inspect:"))
        valuelist = yearlyvalues[monthinput][typeinput]
    else:
        sg.popup("Invalid input. Please try again.")
        return

    if typeinput.lower() == 'needs':
        pielabels = list(needs_list)
    elif typeinput.lower() == 'wants':
        pielabels = list(wants_list)
    elif typeinput.lower() == 'savings':
        pielabels = list(savings_list)
    else:
        sg.popup("Invalid input. Please try again.")
        return
    plt.pie(valuelist, labels=pielabels)
    plt.title(f"{typeinput.capitalize()} Expense Distribution")
    plt.show()

def limit_set():
    global spendinglimit
    spendinglimit = int(sg.popup_get_text("What would you like to set as your spending limit for this month?"))
    sg.popup(f'A spending limit of {spendinglimit} pesos has been set. Note that it can be updated at any time.')

def add_expense():
    newcat = sg.popup_get_text("What is the name of this new spending category?")
    spendtype = sg.popup_get_text("Which type of expenditure is this new category ('needs','wants','savings')?")
    if spendtype == 'needs':
        needs_list.append(newcat)
        monthlyneeds.append(0)
    elif spendtype == 'wants':
        wants_list.append(newcat)
        monthlywants.append(0)
    else:
        savings_list.append(newcat)
        monthlysavings.append(0)
    for i in monthsshown:
        yearlyvalues[i][spendtype].append(0)

def asset_management(df):
    budgeting_option = sg.popup_get_text("Would you want to set your own customized budget or ask for our suggestion (type custom or suggestion)")
    
    if budgeting_option == "custom":
        needs_percentage = int(sg.popup_get_text("What percentage of your current budget do you want allocated for needs? ")) / 100
        wants_percentage = int(sg.popup_get_text("What percentage of your current budget do you want allocated for wants? ")) / 100
        savings_percentage = int(sg.popup_get_text("What percentage of your current budget do you want allocated for savings? ")) / 100
        
        current_budgeting = df['Current_Budget'].iloc[-1]
        sg.popup("Given your current budget, you may choose to spend " + str(current_budgeting * needs_percentage) + " on needs, " + str(current_budgeting * wants_percentage) + " on wants, and save " + str(current_budgeting * savings_percentage))
         
    elif budgeting_option == "suggestion":
        current_budgeting = df['Current_Budget'].iloc[-1]
        sg.popup("We suggest you follow the 50/30/20 rule, you may choose to spend " + str(current_budgeting * 0.5) + " on needs, " + str(current_budgeting * 0.3) + " on wants, and save " + str(current_budgeting * 0.2))
        
    else:
        print("error")

# Function to handle the main program flow
def main():
    set_date()
    try:
        df = pd.read_csv('Financial-Tracker.csv')
    except FileNotFoundError:
        sg.popup('A CSV file is being created for the program...')
        dfcolumns = ['Date', 'Current_Budget', 'Income', 'Expense', 'Needs', 'Wants', 'Savings']
        df = pd.DataFrame(columns=dfcolumns)
        df.to_csv('Financial-Tracker.csv', index=False)
        df = pd.read_csv('Financial-Tracker.csv')
    if len(df['Current_Budget']) == 0:
        setupcounter = False
    else:
        setupcounter = True
    if setupcounter == False:
        set_vars()
        df = setup_budget()
        
    layout = [[sg.Button('Update Data')],
              [sg.Button('Generate Insights')],
              [sg.Button('Set New Goal')],
              [sg.Button('Remove Goal')],
              [sg.Button('View Goals')],
              [sg.Button('View Expenses')],
              [sg.Button('Set Spending Limit')],
              [sg.Button('Add Expense Category')],
              [sg.Button('Asset Management')],
              [sg.Button('Notifications')]]

    window = sg.Window('Financial Tracker', layout)
    
   
    while True:
        
        event, values = window.read()
        if event == sg.WIN_CLOSED:
            break
        if event == 'Update Data':
            df = update_data(df)
            updatemark = True
        if event == 'Asset Management':
            asset_management(df)
        if event == 'Generate Insights':
            generate_insights(df)
        if event == 'Set New Goal':
            saving_goals(goals)
        if event == 'Remove Goal':
            choice_goal_remove(df)
        if event == 'View Goals':
            goal_visual(df)
        if event == 'View Expenses':
            expense_visual()
        if event == 'Set Spending Limit':
            limit_set()
        if event == 'Add Expense Category':
            add_expense()
        if event == 'Notifications':
            notifications(df)
            
    if nowday == monthend:
        monthreset()
    if str(datetime.now())[:10] == f'{nowyear}-12-31':
        yearreset()

    window.close()
    df.to_csv('Financial-Tracker.csv', index=False)

if __name__ == "__main__":
    main()